In [13]:
import pandas as pd


In [14]:
#defince a constant for the sol base unit of 1000000000
SOL_BASE = 1000000000

In [15]:
bare_rarity = pd.read_csv('okay_bears_rarity.csv')

In [16]:
sales = pd.read_csv('transactions.csv')

In [17]:
#convert trade time to datetime from epoch
sales['tradeTime'] = pd.to_datetime(sales['tradeTime'], unit='s')

In [18]:
#divide price by SOL_BASE to get price in SOL
sales['price'] = sales['price'] / SOL_BASE

In [19]:
sales

,Unnamed: 0,tradeCount,tradeTime,price,dex,signature
0,0,2.0,2022-04-28 10:12:39,64.9999,magiceden,55mnGHHCG1zKYUBqW6d3z8D5RKi7wZEA7ksT1LocKTp6Sg...
1,1,4.0,2022-04-28 03:32:02,110.0000,magiceden,JayNLh3WCbdqXfyuYUtWiEsgo6KmsT6SdGSvbdF2x8kqM6...
2,2,1.0,2022-04-27 02:37:46,40.6900,magiceden,2aJvuPkBALGWFiY9wHUHoFcTen4VGARpUgj83q7VaPxfNF...
3,3,7.0,2022-05-02 00:05:41,90.0000,magiceden,3Wsz379AZ6eyCC1crSMSVyUXWE5tTjTvwRY5Zpxn2UFmNa...
4,4,1.0,2022-04-26 20:49:30,10.9900,magiceden,4cSNt1g24CaFeGemPwcpQrn3nKNtPf8ZSWxY8ULSCHepNK...
...,...,...,...,...,...,...
4945,4945,4.0,2022-05-17 14:55:50,230.0000,magiceden,2yfttVJYVxa5ndgPB4XRurWpPNqY5rAeZgR14TrY9rv2pQ...
4946,4946,2.0,2022-05-31 08:03:33,152.6900,magiceden,N3QQeBNWmqh5mLNh3Sn425ecdn4twhnEMiTf9NibAoC3yg...
4947,4947,1.0,2022-05-04 04:40:31,85.0000,magiceden,5Kn15znuwmvpWRvX9Sr2HeRBJdsaGYC3wpWyzcJF59Z2uy...
4948,4948,4.0,2022-05-18 13:14:04,250.0000,magiceden,32UTLsJUX4CVerbUS4BDbXtuxQL6swzKAJ6kCT8A2hMbaM...
